In [1]:
import logging
import time

import numpy as np
from joblib import hash, dump, load
import os
import matplotlib.pyplot as plt
import seaborn as sns

import sys
print(os.path.abspath(''))
from os import path
sys.path.append( path.abspath(os.path.abspath('')) ) 
print(path.abspath(os.path.abspath('')) )

from deer.default_parser import process_args
from microGrid.env.final_env import MyEnv as MG_two_storages_env
import microGrid.experiment.base_controllers as bc
from datetime import datetime
from microGrid.plot_MG_operation import plot_op
import tensorflow as tf
import pandas as pd
from stable_baselines3 import DQN

C:\WorkSpace\code
C:\WorkSpace\code


In [2]:
print("tensorflow work with:", tf.test.gpu_device_name())
logging.basicConfig(level=logging.INFO)
rng = np.random.RandomState()

tensorflow work with: 


In [3]:
class Defaults:
    # ----------------------
    # Experiment Parameters
    # ----------------------
    STEPS_PER_EPOCH = 365*24-1
    EPOCHS = 200
    STEPS_PER_TEST = 365*24-1
    PERIOD_BTW_SUMMARY_PERFS = -1  # Set to -1 for avoiding call to env.summarizePerformance
    
    # ----------------------
    # Environment Parameters
    # ----------------------
    FRAME_SKIP = 1

    # ----------------------
    # DQN Agent parameters:
    # ----------------------
    UPDATE_RULE = 'rmsprop'
    LEARNING_RATE = 0.02
    LEARNING_RATE_DECAY = 0.99
    DISCOUNT = 0.99
    DISCOUNT_INC = 0.99
    DISCOUNT_MAX = 0.98
    RMS_DECAY = 0.9
    RMS_EPSILON = 0.0001
    MOMENTUM = 0
    CLIP_NORM = 1.0
    EPSILON_START = 1.0
    EPSILON_MIN = .1
    EPSILON_DECAY = 2.3e-5
    UPDATE_FREQUENCY = 1
    REPLAY_MEMORY_SIZE = 1000000
    BATCH_SIZE = 32
    FREEZE_INTERVAL = 1000
    DETERMINISTIC = False


In [4]:
# --- Instantiate reward_function ---
price_h2 = 0.1  # 0.1euro/kWh of hydrogen
price_elec_buy = 2.0  # 2euro/kWh

In [5]:
# --- Instantiate environment ---
env = MG_two_storages_env(rng)
absolute_dir = os.path.abspath('')
prod = np.load(absolute_dir + "/microGrid/env/data/BelgiumPV_prod_test.npy")[0:1 * 365 * 24]
cons = np.load(absolute_dir + "/microGrid/env/data/example_nondeterminist_cons_test.npy")[0:1 * 365 * 24]
env_valid = MG_two_storages_env(rng, consumption=cons, production=prod)

reduce_qty_data, length_history, start_history
1 12 0
Sample of the consumption profile (kW): [4.88090280e-05 4.08636652e-04 2.66459683e-03 1.35325348e-02
 5.35289841e-02 1.64925998e-01 3.95869422e-01 7.40576948e-01
 1.08118010e+00 1.23674331e+00 1.12378457e+00 8.51875992e-01
 6.28731540e-01 5.94891051e-01 7.62627204e-01 1.05973501e+00
 1.38386090e+00 1.63210044e+00 1.72495565e+00 1.63169652e+00
 1.38119778e+00 1.04620892e+00 7.09130415e-01 4.30109338e-01]
Min of the consumption profile (kW): 3.569456443385281e-05
Max of the consumption profile (kW): 2.1
Average consumption per day train (kWh): 18.56458726184315
self.production_train brefore
[0. 0. 0. ... 0. 0. 0.]
self.production_train after
[0. 0. 0. ... 0. 0. 0.]
Sample of the production profile (kW): [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.00225469 0.01691014 0.
 0.         0.         0.         0.         0.         0.

C:\WorkSpace\code\venv\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [6]:
env.add_reward("flow_h2", lambda x: x["flow_H2"] * price_h2, 1.)
env.add_reward("buy_energy", lambda x: -x["lack_energy"] * price_elec_buy, 1.)

In [7]:
env_valid.add_reward("flow_h2", lambda x: x["flow_H2"] * price_h2, 1.)
env_valid.add_reward("buy_energy", lambda x: -x["lack_energy"] * price_elec_buy, 1.)

In [8]:
print('MlpPolicy', 
            "learning_rate=",Defaults.LEARNING_RATE, 
            "buffer_size=",Defaults.REPLAY_MEMORY_SIZE, 
            "batch_size=",Defaults.BATCH_SIZE, 
            "gamma=",Defaults.DISCOUNT,
            "exploration_initial_eps=",Defaults.EPSILON_START,
            "exploration_final_eps=",Defaults.EPSILON_MIN,
            "exploration_fraction=",Defaults.EPSILON_DECAY,sep='\n')
model = DQN('MlpPolicy', env, 
            learning_rate=Defaults.LEARNING_RATE, 
            buffer_size=Defaults.REPLAY_MEMORY_SIZE, 
            batch_size=Defaults.BATCH_SIZE, 
            gamma=Defaults.DISCOUNT,
            exploration_initial_eps=Defaults.EPSILON_START,
            exploration_final_eps=Defaults.EPSILON_MIN,
            exploration_fraction=Defaults.EPSILON_DECAY,
            verbose=1)

MlpPolicy
learning_rate=
0.02
buffer_size=
1000000
batch_size=
32
gamma=
0.99
exploration_initial_eps=
1.0
exploration_final_eps=
0.1
exploration_fraction=
2.3e-05
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [9]:
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.evaluation import evaluate_policy

#callback
class VerboseCallback(BaseCallback):
    def _on_step(self) -> bool:
        """
        This method will be called by the model after each call to `env.step()`.

        For child callback (of an `EventCallback`), this will be called
        when the event is triggered.

        :return: (bool) If the callback returns False, training is aborted early.
        """
        return True
    

class BestCallback(BaseCallback):
    def __init__(self, env_valid, patience, dirname, parent_path):
        super().__init__()
        self.validationScores = []
        self.trainScores = []
        self.env_valid = env_valid
        self.bestValidationScoreSoFar = None
        self.cycle = 0
        self.patience = patience
        self.dirname = dirname
        self.parent_path = parent_path
        self.data = None
    
        
    def _on_step(self) -> bool:
        if not all(self.locals["dones"]):
            return True
        self.cycle += 1
        
        mean_reward, std_reward = evaluate_policy(self.model, self.env_valid, n_eval_episodes=1)
        self.validationScores.append(mean_reward)
        
        mean_reward, std_reward = evaluate_policy(self.model, self.training_env, n_eval_episodes=1)
        self.trainScores.append(mean_reward)
        
        # part best
        if self.bestValidationScoreSoFar is None or self.validationScores[-1] > self.bestValidationScoreSoFar:
            self.cycle = 0
            self.bestValidationScoreSoFar = self.validationScores[-1]
            print("new best", self.dirname + "score:"+ str(self.validationScores[-1]))
            self.model.save(self.parent_path + "/" + self.dirname + "/" + self.dirname + 
                            "-score-" + str(self.validationScores[-1]))
            self.data = self.env_valid.get_data()[-2] #-1 empty because env is reset at the end
            #agent.dumpNetwork(self.dirname + "/" + self.dirname + "-score-" + str(validationScores[-1]), epoch)
        if self.cycle >= self.patience:
            return False
        return True
    def get_data(self):
        return self.data
    def get_score(self):
        return self.validationScores, self.trainScores
    

In [ ]:
import time
now = datetime.now()
# dd_mm_YY-H-M-S
dt_string = now.strftime("%d_%m_%Y-%H-%M-%S")
dirname = "result"
filename = "best" + dt_string
best=  BestCallback(env_valid, 220, filename, dirname)
start = time.time()
model.learn(Defaults.EPOCHS * Defaults.STEPS_PER_EPOCH, callback=best)#callback=[verbose_callback, eps_callback, best_callback]
print("time:", time.time() - start)

C:\WorkSpace\code\venv\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


new best best22_04_2022-11-03-26score:-2874.808621198965


C:\WorkSpace\code\venv\lib\site-packages\stable_baselines3\common\save_util.py:276: UserWarning: Path 'result\best22_04_2022-11-03-26' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 8.76e+03  |
|    ep_rew_mean      | -9.71e+03 |
|    exploration_rate | 0.1       |
| time/               |           |
|    episodes         | 4         |
|    fps              | 1474      |
|    time_elapsed     | 23        |
|    total_timesteps  | 35036     |
-----------------------------------
new best best22_04_2022-11-03-26score:-2104.543670715915
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 8.76e+03  |
|    ep_rew_mean      | -8.11e+03 |
|    exploration_rate | 0.1       |
| time/               |           |
|    episodes         | 8         |
|    fps              | 1102      |
|    time_elapsed     | 63        |
|    total_timesteps  | 70072     |
| train/              |           |
|    learning_rate    | 0.02      |
|    loss             | 0.176     |
|    n_updates        | 5017      |
-----------------------------------
-------

In [ ]:
data = best.get_data()
validationScores, trainScores = best.get_score()
print(data.keys())

actions= data["action"]
consumption = data["consumption"]
production = data["production"]
rewards = data["rewards"]
battery_level= data["battery"]
#plot_op(data["action"], data["consumption"], data["production"], data["rewards"], data["battery"], "test.png")
i = 0
plot_op(actions[0+i:100+i],consumption[0+i:100+i],production[0+i:100+i],rewards[0+i:100+i],battery_level[0+i:100+i],"testplot_winter_.png")
plt.show()
i=180*24
plot_op(actions[0+i:100+i],consumption[0+i:100+i],production[0+i:100+i],rewards[0+i:100+i],battery_level[0+i:100+i],"testplot_summer_.png")
plt.show()
i=360*24
plot_op(actions[0+i:100+i],consumption[0+i:100+i],production[0+i:100+i],rewards[0+i:100+i],battery_level[0+i:100+i],"testplot_winter2_.png")
plt.show()

key = "flow_H2"
plt.plot(range(31*24), data[key][:31*24], label=key, color='b')
key = "buy_energy"
plt.plot(range(31*24), data[key][:31*24], label=key, color='r')

plt.legend()
plt.xlabel("Number of hours")
plt.ylabel("Score")
plt.savefig(dirname + "/" + filename + "_plots.pdf")
plt.show()

h = sns.jointplot(x=[battery_level[i] for i in range(len(actions)) if actions[i] == 0],
                  y=[consumption[i] - production[i] for i in range(len(actions)) if actions[i] == 0],
                  kind="hist")
# JointGrid has a convenience function
h.set_axis_labels('charge battery', 'demand', fontsize=16)
plt.savefig(dirname + "/" + filename + "_plots_action0.pdf")
plt.show()

h = sns.jointplot(x=[battery_level[i] for i in range(len(actions)) if actions[i] == 1],
                  y=[consumption[i] - production[i] for i in range(len(actions)) if actions[i] == 1],
                  kind="hist")
# JointGrid has a convenience function
h.set_axis_labels('charge battery', 'demand', fontsize=16)
plt.savefig(dirname + "/" + filename + "_plots_action1.pdf")
plt.show()

h = sns.jointplot(x=[battery_level[i] for i in range(len(actions)) if actions[i] == 2],
                  y=[consumption[i] - production[i] for i in range(len(actions)) if actions[i] == 2],
                  kind="hist")
# JointGrid has a convenience function
h.set_axis_labels('charge battery', 'demand', fontsize=16)
plt.savefig(dirname + "/" + filename + "_plots_action2.pdf")
plt.show()

plt.show()
demande = [consumption[i] - production[i] for i in range(len(actions))]
print("demande moyenne : ", np.mean(demande))
print("demande std : ", np.std(demande))

corr = pd.DataFrame.from_dict(data)
corr = corr.corr()
plt.subplots(figsize=(12, 9))
sns.heatmap(corr, annot = True)
plt.savefig(dirname + "/" + filename + "_heatmap.pdf")
plt.show()
print("reward", np.sum(data["rewards"]))


plt.plot(range(len(validationScores)), validationScores, label="validation score", color='b')
plt.plot(range(len(trainScores)), trainScores, label="train score", color='r')
#plt.plot(x, np.repeat(testScores, nb_rep), label="TS", color='r')
plt.legend()
plt.xlabel("Number of cycle")
plt.ylabel("Score")
plt.savefig(dirname + "/" + filename + "_scores.pdf")
plt.show()